In [2]:
import pandas as pd


def read_excel_file(excel_path, sheet_name):
    xls = pd.ExcelFile(excel_path)
    df = pd.read_excel(xls, sheet_name)
    return df

def convert_to_set(langual_codes, delimiter=' '):
    if isinstance(langual_codes, str):
        return set(langual_codes.split(delimiter))
    else:
        return set() 

excel_file_path = 'data/Frida_5.1_November_2023.xlsx'
df_frida_langal = read_excel_file(excel_file_path, 'Food')
df_frida_langal['LangualCode'] = df_frida_langal['LangualCode'].apply(lambda x: convert_to_set(x, delimiter=','))

df_frida_langal.head(1)

,FødevareNavn,FoodName,FoodID,TaxonomicName,NCBI,FoodEx2Code,FoodEx2Description,FoodOntology,LangualCode,FoodGroupID,FødevareGruppe,FoodGroup
0,"Jordbær, rå","Strawberry, raw",1,Fragaria x ananassa (Duchesne),3747.0,A01EA#F20.A07QT,"Strawberries, PART-CONSUMED-ANALYSED = W/o cap...",FOODON_03301722,"{F0003, K0003, A0833, Z0001, E0150, A0707, A06...",51,Bærfrugt,Soft fruit


In [6]:
# Seperate Cols
df_frida_FoodNameLangual = df_frida_langal[['FoodName', 'LangualCode']]
df_frida_FoodNameLangual

,FoodName,LangualCode
0,"Strawberry, raw","{F0003, K0003, A0833, Z0001, E0150, A0707, A06..."
1,"Apple, raw, all varieties","{F0003, K0003, A0833, Z0001, A0669, E0150, A07..."
2,"Banana, raw","{F0003, K0003, A0833, Z0001, E0150, A0707, A06..."
3,"Potato, raw","{F0003, K0003, A0731, A0829, Z0001, A0706, E01..."
4,"Milk, whole, konventional (not organic), 3.5 %...","{K0003, E0123, A0780, J0135, B1201, C0235, A04..."
...,...,...
1288,"Pineapple, raw",{}
1289,"Grapefruit, raw",{}
1290,"Melon, honeydew, raw",{}
1291,"Kiwi fruit, raw",{}


In [9]:
df_frida_FoodName = df_frida_langal['FoodName']

df_frida_FoodName.to_excel("Preprocess/Frida_Preprocess_FoodNameOnly.xlsx", index=False)

In [22]:
def read_text_file(file_path):
    with open(file_path, 'r', encoding='latin-1') as file:
        lines = file.readlines()
    
    headers = lines[0].strip().split('\t')
    
    data = []
    for line in lines[1:]:
        fields = line.strip().split('\t')
        if len(fields) < len(headers):
            fields += [''] * (len(headers) - len(fields))
        data.append(fields)
    
    df = pd.DataFrame(data, columns=headers)
    return df

text_file_path = 'data/NL RIVM-NEVO 2008-05-22.txt'
df_nevo_langal = read_text_file(text_file_path)
df_nevo_langal['LANGUALCODES'] = df_nevo_langal['LANGUALCODES'].apply(convert_to_set)

df_nevo_langal = df_nevo_langal.rename(columns={'ENGFDNAM':'FoodName'})
df_nevo_FoodName = df_nevo_langal['FoodName']
df_nevo_FoodName.to_excel("Preprocess/Nevo_Preprocess_FoodNameOnly.xlsx", index=False)

# Faiss + Embedding

In [15]:
from openai import OpenAI
import os
import numpy as np
import pandas as pd
import faiss

client = OpenAI()
client.api_key = os.getenv('OPENAI_API_KEY')

In [16]:
# Load and preprocess data
df_Frida_FoodName = pd.read_excel('Preprocess/Frida_Preprocess_FoodNameOnly.xlsx')
df_Nevo_FoodName = pd.read_excel('Preprocess/Nevo_Preprocess_FoodNameOnly.xlsx')
df_Frida_FoodName['FoodName'] = df_Frida_FoodName['FoodName'].astype(str).str.lower().dropna().reset_index(drop=True)
df_Nevo_FoodName['FoodName'] = df_Nevo_FoodName['FoodName'].astype(str).str.lower().dropna().reset_index(drop=True)

In [17]:
def get_embedding(text, model="text-embedding-3-small"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

In [18]:
# Generate embeddings for nevo
df_Nevo_FoodName['Embedding'] = df_Nevo_FoodName['FoodName'].apply(get_embedding)
embeddings = np.vstack(df_Nevo_FoodName['Embedding'].values)

In [20]:
# Create a FAISS index for nevo
index = faiss.IndexFlatL2(embeddings.shape[1])  # L2 distance
index.add(embeddings)

# Save the nevo FAISS index and DataFrame for later use
faiss.write_index(index, 'models/indices/df_Nevo_FoodName_faiss_index.index')
df_Nevo_FoodName.to_pickle('models/embeddings/df_Nevo_FoodName_embeddings.pkl')

In [50]:
import faiss
import numpy as np


# Load FAISS index and DataFrame
index = faiss.read_index('df_Nevo_FoodName_faiss_index.index')
df2 = pd.read_pickle('df_Nevo_FoodName_embeddings.pkl')

def find_similar_food_names(food_name, df2, index, top_n=3):
   # Get the embedding for the food name
    food_embedding = np.array(get_embedding(food_name)).reshape(1, -1)
    
    # Search FAISS index
    _, indices = index.search(food_embedding, top_n)
    
    # Get the similar food names
    similar_food_names = df2.iloc[indices[0]]['FoodName'].tolist()
    
    return similar_food_names


# # Example
# food_name = 'Root vegetables, frozen'
# similar_food_names = find_similar_food_names(food_name, df2, index)
# print(similar_food_names)

['peas and carrots frozen unprepared', 'vegetables mixture for soup frozen', 'vegetables mixture for pea soup frozen']


In [64]:
# Load and preprocess data
df_Frida_FoodName = pd.read_excel('Preprocess/Frida_Preprocess_FoodNameOnly.xlsx')
df_Frida_FoodName['FoodName'] = df_Frida_FoodName['FoodName'].astype(str).str.lower().dropna().reset_index(drop=True)

for idx, food_name in df_Frida_FoodName['FoodName'].items():
    similar_food_names = find_similar_food_names(food_name, df2, index)
    df_Frida_FoodName.at[idx, 'SimilarFoodNames'] = '[' + ', '.join(f'"{name}"' for name in similar_food_names) + ']'
    # print(f"{food_name} : {similar_food_names}")

In [66]:
df_Frida_FoodName

df_Frida_FoodName.to_excel("test_results/Faiss_Embedded_July18/results_testAll_frida_to_nevo.xlsx", index=False)

# Generate Embed for frida, then compare nevo to frida

In [5]:
# Generate embeddings for frida
import faiss
import numpy as np

def get_embedding(text, model="text-embedding-3-small"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

df_Frida_FoodName = pd.read_excel('Preprocess/Frida_Preprocess_FoodNameOnly.xlsx')
df_Frida_FoodName['FoodName'] = df_Frida_FoodName['FoodName'].astype(str).str.lower().dropna().reset_index(drop=True)

df_Frida_FoodName['Embedding'] = df_Frida_FoodName['FoodName'].apply(get_embedding)
embeddings = np.vstack(df_Frida_FoodName['Embedding'].values)

# Create a FAISS index for frida
index = faiss.IndexFlatL2(embeddings.shape[1])  # L2 distance
index.add(embeddings)

# Save the frida FAISS index and DataFrame for later use
faiss.write_index(index, 'df_Frida_FoodName_faiss_index.index')
df_Frida_FoodName.to_pickle('df_Frida_FoodName_embeddings.pkl')

In [14]:
# Save the frida FAISS index and DataFrame for later use
faiss.write_index(index, 'models/indices/df_Frida_FoodName_faiss_index.index')
df_Frida_FoodName.to_pickle('models/embeddings/df_Frida_FoodName_embeddings.pkl')

In [8]:
def find_similar_food_names(food_name, df2, index, top_n=3):
   # Get the embedding for the food name
    food_embedding = np.array(get_embedding(food_name)).reshape(1, -1)
    
    # Search FAISS index
    _, indices = index.search(food_embedding, top_n)
    
    # Get the similar food names
    similar_food_names = df2.iloc[indices[0]]['FoodName'].tolist()
    
    return similar_food_names

# Load and preprocess data
df_Nevo_FoodName = pd.read_excel('Preprocess/Nevo_Preprocess_FoodNameOnly.xlsx')
df_Nevo_FoodName['FoodName'] = df_Nevo_FoodName['FoodName'].astype(str).str.lower().dropna().reset_index(drop=True)

index = faiss.read_index('df_Frida_FoodName_faiss_index.index')
df2 = pd.read_pickle('df_Frida_FoodName_embeddings.pkl')

for idx, food_name in df_Nevo_FoodName['FoodName'].items():
    similar_food_names = find_similar_food_names(food_name, df2, index)
    df_Nevo_FoodName.at[idx, 'SimilarFoodNames'] = '[' + ', '.join(f'"{name}"' for name in similar_food_names) + ']'
    # print(f"{food_name} : {similar_food_names}")

In [9]:
df_Nevo_FoodName

df_Nevo_FoodName.to_excel("test_results/Faiss_Embedded_July18/results_testAll_nevo_to_frida.xlsx", index=False)